# WorkFlow

* Intro to Tensors
* Getting info from Tensors
* Manipulating Tensors
* Tensors and Numpy
* Using @tf.function (speed up regular python fucntion)
* Using GPU/TPU w/TensorFlow
* Excercise


### Intro to `Tensors`

In [148]:
# import tensorflow

import tensorflow as tf
print(tf.__version__)

2.11.0


In [149]:
# creating tensors with tf.contant()
scalar = tf.constant(7)
scalar
# has an empty shape
# tf.constant? ceates a constant tensor from a tensor-like object

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [150]:
# check for number of dimensions
scalar.ndim

0

In [151]:
# create a vector
vector = tf.constant([10,10]) # pass a python list to constant function
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [152]:
# check dimension of vector
vector.ndim

1

In [153]:
# create a matrix
matrix = tf.constant([
    [10,7],
    [7,10]
])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [154]:
# check for dimension
matrix.ndim

2

In [155]:
another_matrix = tf.constant([[1.,4.],
[3.,5.],
[9., 12.]], dtype=tf.float16) # reduce data size occuiance on disk

another_matrix

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[ 1.,  4.],
       [ 3.,  5.],
       [ 9., 12.]], dtype=float16)>

In [156]:
another_matrix.ndim #number of elements in a shape

2

In [157]:
# create another tensor
tensor = tf.constant(
    [[[1,2,3,],
        [4,5,6]],
        [[7,8,9],
        [10,11,12]],
        [[13,14,15],
        [16,17,18]]]
)
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]], dtype=int32)>

In [158]:
tensor.ndim

3

### Creating Tensors with `tf.Variable`

In [159]:
changable_tensor = tf.Variable([10,7])
unchangable_tensor = tf.constant([10,7])

changable_tensor, unchangable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [160]:
# lets chnag one of the elements in a chnagable tensor
# changable_tensor[0] = 13

In [161]:
# trying assign
changable_tensor[1].assign(15)

<tf.Variable 'UnreadVariable' shape=(2,) dtype=int32, numpy=array([10, 15], dtype=int32)>

### Creating Random Tensors

- The neural net always initializes itself with random weights/random tensors
- Adjusts the random weights wrt what the net learns

In [162]:
# create two random tensors
random_1 = tf.random.Generator.from_seed(42) # set the seed for reproducibility
random_1 = random_1.normal(shape=(4,2))
random_1

<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[-0.7565803 , -0.06854702],
       [ 0.07595026, -1.2573844 ],
       [-0.23193763, -1.8107855 ],
       [ 0.09988727, -0.50998646]], dtype=float32)>

In [163]:
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape=(4,2))
random_2

<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[-0.7565803 , -0.06854702],
       [ 0.07595026, -1.2573844 ],
       [-0.23193763, -1.8107855 ],
       [ 0.09988727, -0.50998646]], dtype=float32)>

In [164]:
# with both having a reproducibility value of 42
random_1, random_1, random_1 == random_2 # check hoe they compare

(<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ],
        [ 0.09988727, -0.50998646]], dtype=float32)>,
 <tf.Tensor: shape=(4, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ],
        [ 0.09988727, -0.50998646]], dtype=float32)>,
 <tf.Tensor: shape=(4, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True]])>)

In [165]:
random_2 = tf.random.Generator.from_seed(2) # change the seed value to 2 and see how that compares
random_2 = random_2.normal(shape=(4,2))
random_2

<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[-0.1012345 , -0.2744976 ],
       [ 1.4204658 ,  1.2609464 ],
       [-0.43640924, -1.9633987 ],
       [-0.06452483, -1.056841  ]], dtype=float32)>

In [166]:
random_2, random_1, random_2 == random_1 # change the seed value and the reproducibility is different

(<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
 array([[-0.1012345 , -0.2744976 ],
        [ 1.4204658 ,  1.2609464 ],
        [-0.43640924, -1.9633987 ],
        [-0.06452483, -1.056841  ]], dtype=float32)>,
 <tf.Tensor: shape=(4, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ],
        [ 0.09988727, -0.50998646]], dtype=float32)>,
 <tf.Tensor: shape=(4, 2), dtype=bool, numpy=
 array([[False, False],
        [False, False],
        [False, False],
        [False, False]])>)

### Shuffling: 
* Inheret Order shouldn't affect learning

In [167]:
# not_shuffled = tf.random.Generator.from_seed(42)
# not_shuffled = not_shuffled.normal(shape=(3,2))
# not_shuffled

not_shuffled = tf.constant([[1,2],
[3,4],
[5,6]])
not_shuffled

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[1, 2],
       [3, 4],
       [5, 6]], dtype=int32)>

In [168]:
shuffled = tf.random.shuffle(not_shuffled)
shuffled

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[3, 4],
       [5, 6],
       [1, 2]], dtype=int32)>

In [169]:
not_shuffled, shuffled, not_shuffled == shuffled

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[3, 4],
        [5, 6],
        [1, 2]], dtype=int32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[False, False],
        [False, False],
        [False, False]])>)

In [170]:
# global set_seed
tf.random.set_seed(42) # global
tf.random.shuffle(not_shuffled, seed=42) # operation level random seed
not_shuffled

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[1, 2],
       [3, 4],
       [5, 6]], dtype=int32)>

In [171]:
# create a tensor array of all zeros
tf.zeros(shape=(2,4), dtype=tf.int32)

<tf.Tensor: shape=(2, 4), dtype=int32, numpy=
array([[0, 0, 0, 0],
       [0, 0, 0, 0]], dtype=int32)>

In [172]:
tf.ones([4,6], tf.float64)

<tf.Tensor: shape=(4, 6), dtype=float64, numpy=
array([[1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.]])>

#### Convert a NumPy Array into a Tensor

In [173]:
# import numpy
import numpy as np
numpy_A = np.arange(1,25,dtype=np.int16)
numpy_A

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int16)

In [174]:
# convert the Array into a Tensor
tensor_t = tf.constant(numpy_A)
tensor_t
tensor_tshaped = tf.constant(numpy_A, shape=(2,4,3))
tensor_tshaped

<tf.Tensor: shape=(2, 4, 3), dtype=int16, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6],
        [ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18],
        [19, 20, 21],
        [22, 23, 24]]], dtype=int16)>

In [175]:
numpy_B = np.arange(12.3,31.1, dtype=np.float16)
numpy_B

array([12.3, 13.3, 14.3, 15.3, 16.3, 17.3, 18.3, 19.3, 20.3, 21.3, 22.3,
       23.3, 24.3, 25.3, 26.3, 27.3, 28.3, 29.3, 30.3], dtype=float16)

In [176]:
tensor_tb = tf.constant(numpy_B, dtype=tf.int16)
tensor_tb

<tf.Tensor: shape=(19,), dtype=int16, numpy=
array([12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28,
       29, 30], dtype=int16)>

## Getting information from a tensor.
* Shape 
* Rank
* Axis/Dimension
* Size

In [177]:
# create a rank_4 tensor: rank - no. of tensor dimensions
rank_4_tensor = tf.zeros([2,3,4,5], dtype=tf.int32)
# see how it looks like
rank_4_tensor

<tf.Tensor: shape=(2, 3, 4, 5), dtype=int32, numpy=
array([[[[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]]],


       [[[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]]]], dtype=int32)>

In [178]:
# create a rank_2 tensor: no. of dimension
rank_2_tensor = tf.zeros([3,2], dtype=tf.int16)
rank_2_tensor

<tf.Tensor: shape=(3, 2), dtype=int16, numpy=
array([[0, 0],
       [0, 0],
       [0, 0]], dtype=int16)>

In [179]:
# get the 1st element of the tensor
rank_4_tensor[1]

<tf.Tensor: shape=(3, 4, 5), dtype=int32, numpy=
array([[[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]]], dtype=int32)>

In [180]:
rank_4_tensor.shape

TensorShape([2, 3, 4, 5])

In [181]:
rank_4_tensor.ndim

4

In [182]:
tf.size(rank_4_tensor)

<tf.Tensor: shape=(), dtype=int32, numpy=120>

In [183]:
t = tf.constant([[[1, 1, 1], [2, 2, 2]], [[3, 3, 3], [4, 4, 4]]])

In [184]:
t.shape, t.ndim, tf.size(t)

(TensorShape([2, 2, 3]), 3, <tf.Tensor: shape=(), dtype=int32, numpy=12>)

In [185]:
t

<tf.Tensor: shape=(2, 2, 3), dtype=int32, numpy=
array([[[1, 1, 1],
        [2, 2, 2]],

       [[3, 3, 3],
        [4, 4, 4]]], dtype=int32)>

In [186]:
# deep look into the attributes of our tensor
print("Datatype of Every Element: ", rank_4_tensor.dtype)
print("Number of Dimension (rank): ", rank_4_tensor.ndim)
print("Shape of Tensor: ",rank_4_tensor.shape)

Datatype of Every Element:  <dtype: 'int32'>
Number of Dimension (rank):  4
Shape of Tensor:  (2, 3, 4, 5)


In [187]:
print("Elements along the 0 axis: ",rank_4_tensor.shape[0])

Elements along the 0 axis:  2


In [188]:
print("Elements along the last axis: ", rank_4_tensor.shape[-1])

Elements along the last axis:  5


In [189]:
print("Total number of elements in our tensor: ",tf.size(rank_4_tensor).numpy())

Total number of elements in our tensor:  120


#### Indexing Tensors

In [190]:
#python list and indexing
values_to_index = [1,2,3,4,5]
# get the first three elements in the list
values_to_index[:3]

[1, 2, 3]

In [191]:
# get the first two elements of each dimension
rank_4_tensor[:2,:3,:1,:]

<tf.Tensor: shape=(2, 3, 1, 5), dtype=int32, numpy=
array([[[[0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0]]],


       [[[0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0]]]], dtype=int32)>

> Adding an Extra Dimension into a Tensor

In [192]:
# create a rank 2 tensor
rank_2_tensor = tf.constant([[5,7],
[9,2]])
rank_2_tensor.shape, rank_2_tensor.ndim

(TensorShape([2, 2]), 2)

In [193]:
# get the last item [-1] of each of the dimensions
rank_2_tensor[:,-1]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([7, 2], dtype=int32)>

In [194]:
# create a rank_3 out of rank_2 tensor
rank_3_tensor = rank_2_tensor[..., tf.newaxis]
rank_3_tensor

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[5],
        [7]],

       [[9],
        [2]]], dtype=int32)>

In [195]:
rank_2_tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[5, 7],
       [9, 2]], dtype=int32)>

In [196]:
#expand using another method
tf.expand_dims(rank_2_tensor, axis=-1) #expnads the last axis of the rank_2_tensor

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[5],
        [7]],

       [[9],
        [2]]], dtype=int32)>

In [197]:
# expand from the front
tf.expand_dims(rank_2_tensor, axis=0)

<tf.Tensor: shape=(1, 2, 2), dtype=int32, numpy=
array([[[5, 7],
        [9, 2]]], dtype=int32)>

In [198]:
# expand from the middlw
tf.expand_dims(rank_2_tensor, axis=1)#.shape.as_list()

<tf.Tensor: shape=(2, 1, 2), dtype=int32, numpy=
array([[[5, 7]],

       [[9, 2]]], dtype=int32)>

#### Operations on Tensors, Manipulating Tensors

**Basic Operations : ADMS** : `Element-wise ops`

In [199]:
tensor = tf.constant([[6.3,7.8],
[2,3]], dtype=tf.float16)
tensor_A = tensor * 10
tensor_A 

<tf.Tensor: shape=(2, 2), dtype=float16, numpy=
array([[63., 78.],
       [20., 30.]], dtype=float16)>

Its a Computing advantage when we use tf's built in functions for numeric ops

In [200]:
tf_built = tf.subtract(tensor_A, 12.4) # dtypes must be the same
tf_built

<tf.Tensor: shape=(2, 2), dtype=float16, numpy=
array([[50.6, 65.6],
       [ 7.6, 17.6]], dtype=float16)>

#### **Matrix Multiplication**

In [201]:
# use the matmul
tensor

<tf.Tensor: shape=(2, 2), dtype=float16, numpy=
array([[6.3, 7.8],
       [2. , 3. ]], dtype=float16)>

In [202]:
check_shape = tf.matmul(tensor, tensor_A)
check_shape.shape

TensorShape([2, 2])

In [203]:
check_shape

<tf.Tensor: shape=(2, 2), dtype=float16, numpy=
array([[553. , 725.5],
       [186. , 246. ]], dtype=float16)>

In [204]:
# matrix multiplication with Python
check_shape_1 = tensor_A @ tensor
check_shape_1, check_shape, check_shape_1 == check_shape

(<tf.Tensor: shape=(2, 2), dtype=float16, numpy=
 array([[553. , 725.5],
        [186. , 246. ]], dtype=float16)>,
 <tf.Tensor: shape=(2, 2), dtype=float16, numpy=
 array([[553. , 725.5],
        [186. , 246. ]], dtype=float16)>,
 <tf.Tensor: shape=(2, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True]])>)

In [205]:
# create a [3,2] matrix

X = tf.constant([[1,2],[3,4],[5,6]]).numpy()

Y = tf.constant([[7,8],[9,10],[11,12]]).numpy()

X, Y

(array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32),
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]], dtype=int32))

In [206]:
# matrix multiply X and Y
# X @ Y
# tf.matmul(X,Y)

# Both multiplications fuction throw in an error : Matrix size-incompatible:


In [207]:
# Y = tf.transpose(Y)
# tf.matmul(X,Y)

In [208]:
Y

array([[ 7,  8],
       [ 9, 10],
       [11, 12]], dtype=int32)

In [209]:
# X = tf.transpose(X)
# tf.matmul(X,Y)

In [210]:
X,Y

(array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32),
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]], dtype=int32))

In [211]:
# perfome matrix multiplication btn X and Y (transposed)
tf.matmul(X, tf.transpose(Y))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]], dtype=int32)>

In [212]:
# perfom matrix multiplication btn X and Y (reshaped)
tf.matmul(X, tf.reshape(Y, shape=(2,3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [214]:
# check the difference btn reshaped and transposed and normal Y

# normal Y
print("Normal Y: ")
print(Y, "\n")

print("Reshaped Y: ")
print(tf.reshape(Y, shape=(2,3)), "\n")

print("Transposed Y: ")
print(tf.transpose(Y), "\n")



Normal Y: 
[[ 7  8]
 [ 9 10]
 [11 12]] 

Reshaped Y: 
tf.Tensor(
[[ 7  8  9]
 [10 11 12]], shape=(2, 3), dtype=int32) 

Transposed Y: 
tf.Tensor(
[[ 7  9 11]
 [ 8 10 12]], shape=(2, 3), dtype=int32) 

